In [1]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from torch import nn
from torch.nn import BCELoss
from torch.optim import SGD, Adam
from keras_preprocessing.text import Tokenizer 
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tqdm import tqdm
import tqdm
import time

2024-03-22 12:45:32.354191: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-22 12:45:32.415156: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-22 12:45:33.507426: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
class TextCodeDataset(Dataset):
    def __init__(self, data_path, pad_seq_len, use_AST=True, classify_btype=True):
        self.data_path = data_path
        self.pad_seq_len = pad_seq_len
        self.use_AST, self.classify_btype = use_AST, classify_btype
        self.data_pd = pd.read_csv(data_path,
                                   error_bad_lines=False, index_col=False, dtype='unicode', encoding='latin-1',
                                   low_memory=False).sample(frac=1)
        self.y_dev, self.y_btype = list(self.data_pd['FixedByID']), list(self.data_pd['Name'])
        self.y_dev, self.y_btype = [str(x) for x in self.y_dev], [str(x) for x in self.y_btype]
        self.y_dev, self.y_btype = [x.split('|') for x in self.y_dev], [x.split('|') for x in self.y_btype]

        self.x_context, self.x_AST = list(self.data_pd['Title_Description']), list(self.data_pd['AST'])
        self.x_context, self.x_AST = [str(x) for x in self.x_context], [str(x) for x in self.x_AST]

    def tokenize_input(self, tokenizer_C: Tokenizer, tokenizer_A: Tokenizer):
        self.x_context = tokenizer_C.texts_to_sequences(self.x_context)
        self.x_AST = tokenizer_A.texts_to_sequences(self.x_AST)
        self.x_context = pad_sequences(self.x_context, maxlen=self.pad_seq_len, padding='post')
        self.x_AST = pad_sequences(self.x_AST, maxlen=self.pad_seq_len, padding='post')
        self.x_context = torch.from_numpy(self.x_context)
        self.x_AST = torch.from_numpy(self.x_AST)

    def map_output(self, map_d: dict, map_b: dict):
        tensor_d, tensor_b = torch.zeros((len(self.data_pd), len(map_d))), torch.zeros((len(self.data_pd), len(map_b)))
        for i, ds in enumerate(self.y_dev):
            for d in ds:
                tensor_d[i][map_d[d]] = 1
        for i, bs in enumerate(self.y_btype):
            for b in bs:
                tensor_b[i][map_b[b]] = 1
        self.y_dev, self.y_btype = tensor_d, tensor_b

    def __getitem__(self, i):
        input = self.x_context[i], self.x_AST[i] if self.use_AST else self.x_context[i]
        output = self.y_dev[i], self.y_btype[i] if self.classify_btype else self.y_dev[i]
        return input, output

    def __len__(self):
        return len(self.data_pd)

In [3]:
def tokenize_dataset_input(train_dataset: TextCodeDataset, test_dataset: TextCodeDataset):
    tokenizer_C = Tokenizer()
    tokenizer_C.fit_on_texts(train_dataset.x_context + test_dataset.x_context)
    tokenizer_A = Tokenizer()
    tokenizer_A.fit_on_texts(train_dataset.x_AST + test_dataset.x_AST)
    train_dataset.tokenize_input(tokenizer_C, tokenizer_A)
    test_dataset.tokenize_input(tokenizer_C, tokenizer_A)
    return [len(tokenizer_C.word_index)+100, len(tokenizer_A.word_index)+100]


def map_dataset_output(train_dataset: TextCodeDataset, test_dataset: TextCodeDataset):
    set_d, set_b = set(), set()
    for i in range(len(train_dataset)):
        for d in train_dataset.y_dev:
            set_d.update(d)
        for b in train_dataset.y_btype:
            set_b.update(b)
    for i in range(len(test_dataset)):
        for d in test_dataset.y_dev:
            set_d.update(d)
        for b in test_dataset.y_btype:
            set_b.update(b)
    labels_d, labels_b = list(set_d), list(set_b)
    labels_d.sort(), labels_b.sort()
    map_d, map_b = {labels_d[i]: i for i in range(len(labels_d))}, {labels_b[i]: i for i in range(len(labels_b))}
    train_dataset.map_output(map_d, map_b)
    test_dataset.map_output(map_d, map_b)
    return labels_d, labels_b

In [4]:
# dataset
# train_path = 'Data/powershell/C_uA_Train.csv'
train_path = 'Data/aspnet/trainC40.csv'
test_path = 'Data/aspnet/Nov10数据/IssueaspnetcoreWebScraptestdata5.csv'
MAX_SEQ_LEN = 300
EMB_DIM = 100
Learning_Rate = 6e-3
EPOCH = 300
device = 'cuda' if torch.cuda.is_available() else 'cpu'
Weight_MSE, Weight_BCE = 0, 1
# 0, 1.0, 1e-3, 100 -> 0.008
# 0.1, 0.9, 1e-3, 100 -> 0.04
# 0.2, 0.8, 1e-3, 100 -> 0.06
# 0.3, 0.7, 1e-3, 100 -> 0.12
# 0.4, 0.6, 1e-3, 100 -> 0.15
# 0.5, 0.5, 1e-3, 100 -> 0.19
# 0.6, 0.4, 1e-3, 100 -> 0.2
# 0.7, 0.3, 1e-3, 100 -> 0.2, stable
# 0.8, 0.2, 1e-3, 100 -> 0.2, convergence   | 0.07 simple model
# 0.9, 0.1, 1e-3, 100 -> 0.15, convergence
# 1, 0, 1e-3, 100 -> ~0
train_dataset = TextCodeDataset(train_path, pad_seq_len=MAX_SEQ_LEN)
test_dataset = TextCodeDataset(test_path, pad_seq_len=MAX_SEQ_LEN)

vocab_size = tokenize_dataset_input(train_dataset, test_dataset)
idx2label = map_dataset_output(train_dataset, test_dataset)
num_out = [len(x) for x in idx2label]

# dataloader
train_loader = DataLoader(train_dataset, batch_size=64)
test_loader = DataLoader(test_dataset, batch_size=64)

In [5]:
print(len(train_loader))
for data in train_loader:
    (a, b), (c, d) = data
    print(a.shape)
    print(b.shape)
    print(c.shape)
    print(d.shape)
    (a, b), (c, d) = train_dataset[0]
    print(a.shape)
    print(b.shape)
    print(c.shape)
    print(d.shape)
    break

27
torch.Size([64, 300])
torch.Size([64, 300])
torch.Size([64, 71])
torch.Size([64, 176])
torch.Size([300])
torch.Size([300])
torch.Size([71])
torch.Size([176])


In [6]:
def testnan(name, x):
    if torch.isnan(x).any():
        print(name, x)

def repnan(x):
    return torch.where(torch.isnan(x), torch.ones_like(x), x)

In [7]:
#TODO :训练的太快了，是不是参数太少了拟合不了？
# class MetaModel(nn.Module):
#     def __init__(self, vocab_size: list, emb_dim: int, seq_len: int, num_out: list):
#         """
#         Args:
#             vocab_size: list, the size of C/A vocab
#             emb_dim: : int, the dim of C&A emb layer
#             seq_len:  MAX_SEQ_LEN
#             num_out: list, the output size of D/B
#         """
#         super(MetaModel, self).__init__()
#         self.vocab_size = vocab_size
#         self.emb_dim = emb_dim
#         self.seq_len = seq_len
#         self.num_out = num_out

#         self.context = nn.Sequential(
#             nn.Embedding(self.vocab_size[0], self.emb_dim),
#             nn.Conv1d(self.seq_len, 64, kernel_size=2),
#             nn.ReLU(),
#             nn.Conv1d(64, 128, kernel_size=2),
#             nn.ReLU(),
#             nn.MaxPool1d(50, 1),
#             nn.Flatten(),
#         )
#         self.AST = nn.Sequential(
#             nn.Embedding(self.vocab_size[1], self.emb_dim),
#             nn.Conv1d(self.seq_len, 50, kernel_size=2),
#             nn.ReLU(),
#             nn.Conv1d(50, 100, kernel_size=2),
#             nn.ReLU(),
#             nn.MaxPool1d(50, 1),
#             nn.Flatten(),
#         )
#         self.fc = nn.Sequential(
#             nn.BatchNorm1d(11172, affine=False),
#             nn.Dropout(0.5),
#             nn.Linear(11172, 5000),
#             nn.ReLU(),
#             nn.Dropout(0.5),
#             nn.Linear(5000, 1000),
#             nn.ReLU(),
#             nn.Dropout(0.5),
#             nn.Linear(1000, 500),
#             nn.ReLU(),
#         )
#         self.fc_d = nn.Sequential(
#             nn.Linear(500, self.num_out[0]),
#             # nn.Softmax(dim=1),
#             # nn.Sigmoid(),
#         )

#         self.fc_b = nn.Sequential(
#             nn.Linear(500, self.num_out[1]),
#             # nn.Softmax(dim=1),
#             # nn.Sigmoid(),
#         )

#     def forward(self, x_C, x_A):
#         # x, y
#         x_C = self.context(x_C) 
#         x_A = self.AST(x_A)
#         x = torch.concat((x_C, x_A), 1)
#         x = self.fc(x)
#         y_d = self.fc_d(x) 
#         y_b = self.fc_b(x) 
#         return y_d, y_b 
class MetaModel(nn.Module):
    def __init__(self, vocab_size: list, emb_dim: int, seq_len: int, num_out: list):
        """
        Args:
            vocab_size: list, the size of C/A vocab
            emb_dim: : int, the dim of C&A emb layer
            seq_len:  MAX_SEQ_LEN
            num_out: list, the output size of D/B
        """
        super(MetaModel, self).__init__()
        self.vocab_size = vocab_size
        self.emb_dim = emb_dim
        self.seq_len = seq_len
        self.num_out = num_out

        self.context = nn.Sequential(
            nn.Embedding(self.vocab_size[0], self.emb_dim),
            nn.Conv1d(self.seq_len, 64, kernel_size=2),
            nn.ReLU(),
            nn.MaxPool1d(99, 1),
            nn.Flatten(),
        )
        self.AST = nn.Sequential(
            nn.Embedding(self.vocab_size[1], self.emb_dim),
            # nn.LSTM(self.emb_dim, 25, batch_first=True, bidirectional=True),
            # nn.MaxPool1d(50, 1),
            nn.Conv1d(self.seq_len, 50, kernel_size=2),
            nn.ReLU(),
            nn.MaxPool1d(99, 1),
            nn.Flatten(),
        )
        self.fc = nn.Sequential(
            nn.BatchNorm1d(114, affine=False), # one head
            nn.Dropout(0.5),
            nn.Linear(114, 50), # one head
            nn.ReLU()
        )
        self.fc_d = nn.Sequential(
            nn.Linear(50, self.num_out[0]),
            nn.Sigmoid(),
        )

        self.fc_b = nn.Sequential(
            nn.Linear(50, self.num_out[1]),
            nn.Sigmoid(),
        )

    def forward(self, x_C, x_A):
        # x, y
        x_C = self.context(x_C) 
        x_A = self.AST(x_A) # one head
        x = torch.concat((x_C, x_A), 1) # one head
        x = self.fc(x) # one head
        y_d = self.fc_d(x) # one tail 
        y_b = self.fc_b(x) # one tail 
        return y_d, y_b # one tail

In [8]:
N_STEPS = len(train_loader) * EPOCH
def get_lr_scheduler(optimizer):
    return torch.optim.lr_scheduler.OneCycleLR(
        optimizer=optimizer,
        max_lr=Learning_Rate,
        total_steps=N_STEPS,
        pct_start=0.10,
        anneal_strategy='cos',
        div_factor=1e3,
        final_div_factor=1e4,
    )


In [9]:
# model
model = MetaModel(vocab_size, EMB_DIM, MAX_SEQ_LEN, num_out).to(device)

# loss_func
def loss_fn(y, y_pred):
    return -torch.sum(0.8*y*torch.log(y_pred) + 0.2*(1-y)*torch.log(1-y_pred))
# loss_fn = torch.nn.BCEWithLogitsLoss().to(device)
# loss_fn_mse = torch.nn.MSELoss().to(device)
# loss_fn_ce = torch.nn.MultiLabelSoftMarginLoss().to(device)
# loss_fn_ce = torch.nn.BCEWithLogitsLoss().to(device)
# loss_name = 'BCELogits_Adam'
loss_name = 'BCELogits'

# Optimizer
optimizer = torch.optim.Adam(
    params=model.parameters(),
    lr=Learning_Rate,
)

# Learning Rate Scheduler
LR_SCHEDULER = get_lr_scheduler(optimizer)

In [10]:
def metrics(y: torch.Tensor, y_pred: torch.Tensor, threshold: float = 0.5, from_logits = False, split_pos = num_out):
    if from_logits == True:
        y_pred = nn.Sigmoid()(y_pred)
    pred = torch.where(y_pred > threshold, 1, 0)
    
    y_d, y_b = torch.split(y, split_pos, dim=1)
    pred_d, pred_b = torch.split(pred, split_pos, dim=1)
    
    TPd, TPb = torch.sum(y_d * pred_d, dim=1), torch.sum(y_b * pred_b, dim=1)
    TNd, TNb = torch.sum((1-y_d) * (1-pred_d), dim=1), torch.sum((1-y_b) * (1-pred_b), dim=1)
    FPd, FPb = torch.sum((1-y_d) * pred_d , dim=1), torch.sum((1-y_b) * pred_b , dim=1)
    FNd, FNb = torch.sum(y_d * (1-pred_d), dim=1), torch.sum(y_b * (1-pred_b), dim=1)

    acc = torch.mean((TPd + TNd) / (TPd + TNd + FPd + FNd + 1e-6)).item(), torch.mean((TPb + TNb) / (TPb + TNb + FPb + FNb + 1e-6)).item()
    recall = torch.mean(TPd / (TPd + FNd + 1e-6)).item(), torch.mean(TPb / (TPb + FNb + 1e-6)).item()
    precision = torch.mean(TPd / (TPd + FPd + 1e-6)).item(), torch.mean(TPb / (TPb + FPb + 1e-6)).item()
    F1 = 2 * recall[0] * precision[0] / (recall[0] + precision[0] + 1e-6), 2 * recall[1] * precision[1] / (recall[1] + precision[1] + 1e-6)
    
    return {
        'acc': acc,
        'precision': precision,
        'recall': recall,
        'F1': F1
    }

def one_forward(data):
    (x_context, x_AST), (y_dev, y_btype) = data
    y_dev_pred, y_btype_pred = model(x_context.long().to(device), x_AST.long().to(device)) 
    
    y = torch.concat((y_dev, y_btype), 1).to(device) 
    y_pred = torch.concat((y_dev_pred, y_btype_pred), 1) 

    # loss = Weight_MSE * loss_fn_mse(y, nn.Sigmoid()(y_pred)) + Weight_BCE * loss_fn_ce(y, y_pred)
    loss = loss_fn(y, y_pred)
    metric = metrics(y, y_pred, from_logits=False)
    
    return loss, metric['acc'], metric['precision'], metric['recall'], metric['F1']


def one_backward(optimizer, loss):
    optimizer.zero_grad()
    loss.backward()
    # Gradient clipping
    torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
    optimizer.step()

def one_train_scaler(scaler, data, optimizer):
    with torch.cuda.amp.autocast():
        optimizer.zero_grad()
        loss, acc, precision, recall, f1 = one_forward(data)
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

In [11]:
print(num_out)

[71, 176]


In [12]:
from torch.utils.tensorboard import SummaryWriter
# torch.autograd.set_detect_anomaly(True)
# scaler = torch.cuda.amp.GradScaler()

writer = SummaryWriter('../../tf-logs')
for epoch in tqdm.trange(EPOCH):

    # train
    model.train()
    for step, data in enumerate(train_loader):

        loss, acc, precision, recall, f1 = one_forward(data)
        one_backward(optimizer, loss)
        
        # one_train_scaler(scaler, data, optimizer)
        # LR_SCHEDULER.step()
        

    # val
    model.eval()
    val_loss, val_acc, val_precision, val_recall, val_F1 = 0, [0, 0], [0, 0], [0, 0], [0, 0]
    with torch.no_grad():
        for step, data in enumerate(test_loader):
            loss, acc, precision, recall, f1 = one_forward(data)
            l = len(test_loader)
            val_loss += loss.item() / l
            val_acc[0] += acc[0]/l
            val_acc[1] += acc[1]/l
            val_precision[0] += precision[0]/l
            val_precision[1] += precision[1]/l
            val_recall[0] += recall[0]/l
            val_recall[1] += recall[1]/l
            val_F1[0] += f1[0]/l 
            val_F1[1] += f1[1]/l 
            
    print('{}th epoch\n val_loss: {}\n val_acc:{}\n val_precision:{}\n val_recall:{}\n val_f1: {}'.format(epoch, val_loss, val_acc, val_precision, val_recall, val_F1))
    writer.add_scalar('val_loss'+loss_name+str(Learning_Rate), val_loss, epoch)
    writer.add_scalar('val_acc_d'+loss_name+str(Learning_Rate), val_acc[0], epoch)
    writer.add_scalar('val_acc_b'+loss_name+str(Learning_Rate), val_acc[1], epoch)
    # writer.add_scalar('val_precision'+loss_name+str(Learning_Rate), val_precision, epoch)
    # writer.add_scalar('val_recall'+loss_name+str(Learning_Rate), val_recall, epoch)
    writer.add_scalar('val_f1_d'+loss_name+str(Learning_Rate), val_F1[0], epoch)
    writer.add_scalar('val_f1_b'+loss_name+str(Learning_Rate), val_F1[1], epoch)

    # save model
    # if epoch % 20 == 0:
        # torch.save(model, '../savedmodel/model_epoch{}.pth'.format(epoch))

writer.close()

  0%|          | 1/300 [00:01<06:52,  1.38s/it]

0th epoch
 val_loss: 2229.163038170856
 val_acc:[0.485934625501218, 0.5542751783910004]
 val_precision:[0.009072535306863163, 0.014598334086653978]
 val_recall:[0.3297818847324536, 0.38969035641006805]
 val_f1: [0.017658836956747057, 0.02813688790075832]


  1%|          | 2/300 [00:01<03:43,  1.33it/s]

1th epoch
 val_loss: 2230.5786398182745
 val_acc:[0.49199133852253785, 0.5555679150249646]
 val_precision:[0.00859973553325171, 0.016157226758482662]
 val_recall:[0.3094014706818954, 0.42851272095804627]
 val_f1: [0.016733611686328333, 0.031135189119971412]


  1%|          | 3/300 [00:01<02:43,  1.82it/s]

...

297th epoch
 val_loss: 658.0621430770211
 val_acc:[0.9865652504174606, 0.9784630070561949]
 val_precision:[0.5699105781057606, 0.10360250796150904]
 val_recall:[0.687931654245957, 0.07356817113316577]
 val_f1: [0.6231101494363164, 0.08477887895115693]


100%|█████████▉| 299/300 [01:30<00:00,  3.21it/s]

298th epoch
 val_loss: 656.7053594174591
 val_acc:[0.9870332298071489, 0.9784823085950769]
 val_precision:[0.5827872856803561, 0.10360250796150904]
 val_recall:[0.687931654245957, 0.07356817113316577]
 val_f1: [0.6307497937663284, 0.08477887895115693]


100%|██████████| 300/300 [01:30<00:00,  3.31it/s]

299th epoch
 val_loss: 655.744819972826
 val_acc:[0.9874829364859539, 0.9784977461980737]
 val_precision:[0.5956948814184769, 0.10414598570407735]
 val_recall:[0.687931654245957, 0.07311527366223543]
 val_f1: [0.6383088652181371, 0.0846576630661553]


In [13]:
(x_c, x_a), (y_d, y_b) = test_dataset[0]
x_c = torch.reshape(x_c.long().to(device), (1, 300))
x_a = torch.reshape(x_a.long().to(device), (1, 300))
y_d_pred, y_b_pred = model(x_c, x_a)

print(y_d_pred, y_b_pred)
# y_d_pred, y_b_pred = nn.Softmax(dim=1)(y_d_pred), nn.Sigmoid()(y_b_pred)
print(y_d_pred[0][-29])
#TODO: 调整阈值
#TODO: 多标签分类问题：换用F1分数、Hamming损失或Jaccard相似度
print(torch.where(y_d_pred > 0.5, 1.0, 0.0), torch.where(y_b_pred > 0.5, 1.0, 0.0))


tensor([[0.0696, 0.0710, 0.0313, 0.0568, 0.0452, 0.1033, 0.0881, 0.0791, 0.0696,
         0.0924, 0.1817, 0.0232, 0.0103, 0.0561, 0.0231, 0.0218, 0.0134, 0.1286,
         0.1104, 0.0990, 0.0407, 0.0341, 0.0852, 0.1356, 0.0344, 0.0875, 0.0483,
         0.0324, 0.1097, 0.0123, 0.0949, 0.0242, 0.0643, 0.0125, 0.0753, 0.0149,
         0.0416, 0.0541, 0.0656, 0.1316, 0.1336, 0.0228, 0.4929, 0.0225, 0.0754,
         0.0328, 0.0635, 0.0188, 0.0494, 0.2211, 0.0706, 0.0359, 0.0401, 0.0213,
         0.0822, 0.0477, 0.0665, 0.0179, 0.0140, 0.0629, 0.0417, 0.0086, 0.0965,
         0.0872, 0.1246, 0.0793, 0.0466, 0.0763, 0.8481, 0.2071, 0.1395]],
       device='cuda:0', grad_fn=<SigmoidBackward0>) tensor([[0.0265, 0.0769, 0.0734, 0.0278, 0.0504, 0.1285, 0.2128, 0.0731, 0.0296,
         0.0295, 0.0297, 0.1111, 0.0318, 0.3357, 0.0444, 0.2317, 0.1443, 0.0762,
         0.1007, 0.0208, 0.0520, 0.0660, 0.0669, 0.0336, 0.0208, 0.0895, 0.0440,
         0.0215, 0.0829, 0.1820, 0.0813, 0.2198, 0.0889, 0.3523

In [14]:
print(y_d, y_b)

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.]) tensor([0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 1., 0.,

In [15]:
y_d = y_d.reshape(1, -1)
y_d_pred = y_d_pred.reshape(1, -1)
for i, l in metrics(y_d.to(device), y_d_pred, from_logits = True, split_pos = num_out[0]).items():
    print(i, l)

ValueError: not enough values to unpack (expected 2, got 1)

In [ ]:
y_b = y_b.reshape(1, -1)
y_b_pred = y_b_pred.reshape(1, -1)
metrics(y_b.to(device), y_b_pred, from_logits = True), metrics(y_b.to(device), y_b_pred, from_logits = False), metricsy_b.to(device), y_b_pred, from_logits = True, recall=True), metricsy_b.to(device), y_b_pred, from_logits = False, recall=True)

In [ ]:
print(num_out)